# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

In [1]:
!pip install gensim

import numpy
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [2]:
import numpy as np

v_size = len(model.key_to_index)  #model.key_to_index : 単語を数字に変換するための辞書   v_size : 300万単語
d_emb = model.vector_size #d_emb : 300次元 [1,2,3,...,300]
E = np.zeros((v_size+1, d_emb)) #300次元が縦に300万1個ある

id_to_word = {0:"<PAD>"}
word_to_id = {"<PAD>":0}

for i,word in enumerate(model.key_to_index,1):
  E[i] = model[word]  #model[word] : 単語をベクトルに変換する
  id_to_word[i] = word  #{i:word}
  word_to_id[word] = i  #{word:i}

## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [3]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

--2025-05-20 14:03:12--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.102, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7439277 (7.1M) [application/zip]
Saving to: ‘SST-2.zip’

SST-2.zip           100%[===================>]   7.09M  --.-KB/s    in 0.06s   

2025-05-20 14:03:12 (109 MB/s) - ‘SST-2.zip’ saved [7439277/7439277]

Archive:  SST-2.zip
   creating: SST-2/
  inflating: SST-2/dev.tsv           
   creating: SST-2/original/
  inflating: SST-2/original/README.txt  
  inflating: SST-2/original/SOStr.txt  
  inflating: SST-2/original/STree.txt  
  inflating: SST-2/original/datasetSentences.txt  
  inflating: SST-2/original/datasetSplit.txt  
  inflating: SST-2/original/dictionary.txt  
  inflating: SST-2/original/original_rt_snippets.txt  
  inflating: SST-2/ori

In [4]:
import pandas as pd
import torch

train_df = pd.read_csv("SST-2/train.tsv", sep="\t")
dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")


def word_to_id_change(text, word_to_id):
  list = []
  for word in text.split():
    if word in word_to_id:
      list.append(word_to_id[word])
  return list

def make_dataset(df, word_to_id):
  dataset = []
  for _,row in df.iterrows():
    id = word_to_id_change(row["sentence"], word_to_id)
    if len(id) == 0:
      continue
    data = {"text": row["sentence"], "label": torch.tensor([float(row["label"])]), "input_ids": torch.tensor(id)}
    dataset.append(data)

  return dataset


train_dataset = make_dataset(train_df, word_to_id)
dev_dataset = make_dataset(dev_df, word_to_id)

## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [5]:
import torch.nn as nn

def average_bekutoru(ids, E):
  sum = []
  for i in ids:
    sum.append(E[i.item()])
  return torch.mean(torch.tensor(sum), dim=0).float()


class SentimentClassifier(nn.Module):
    def __init__(self, embedding_dim):
        super(SentimentClassifier, self).__init__()
        self.linear = nn.Linear(embedding_dim, 1)  # w^T x + b
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)      # shape: (batch_size, 1)
        out = self.sigmoid(out)   # shape: (batch_size, 1)
        return out

print(average_bekutoru(train_dataset[0]["input_ids"],E))

tensor([ 0.5841, -0.1052, -0.0329, -0.1955,  0.1888,  0.4000, -0.1472, -0.2301,
         0.2297, -0.1309,  0.3233,  0.1323,  0.2015, -0.1077, -0.0163,  0.1709,
        -0.3846, -0.1186, -0.1999, -0.1891,  0.1664,  0.0469,  0.3069, -0.1310,
        -0.4717, -1.0281, -0.1221, -0.0110,  0.2898, -0.2106,  2.6787,  0.1628,
        -0.1114, -0.4462,  0.1151,  0.2763, -0.0297, -0.4203,  0.1303, -0.1559,
         0.0170, -0.0847,  0.1870,  0.3271, -0.0883, -0.2434,  0.0543, -0.0810,
        -0.1885, -0.2166])


<ipython-input-5-91d45d5908c9>:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.mean(torch.tensor(sum), dim=0).float()


## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [6]:
import torch.optim as optim

def train(model, dataset, E, epochs=5, batch_size=16, lr=0.001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for i in range(0, len(dataset), batch_size):
            batch = dataset[i:i+batch_size]

            feature_vectors = []
            labels = []

            for example in batch:
                avg_vec = average_bekutoru(example["input_ids"], E)
                feature_vectors.append(avg_vec)
                labels.append(example["label"].item())

            inputs = torch.stack(feature_vectors)             # shape: (batch_size, embedding_dim)
            targets = torch.tensor(labels, dtype=torch.float) # shape: (batch_size,)

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()                 # shape: (batch_size,)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (len(dataset) // batch_size + 1)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

# --------------------
# 実行例（データとモデルの前提があるとする）
# E: 事前学習済み埋め込み行列（torch.FloatTensor）
# train_dataset: [{"input_ids": tensor([...]), "label": tensor([0.0 or 1.0])}, ...]

embedding_dim = E.shape[1]
model = SentimentClassifier(embedding_dim)

# 学習実行
train(model, train_dataset, E, epochs=10, batch_size=16)

Epoch 1/10 - Loss: 0.5540
Epoch 2/10 - Loss: 0.5094
Epoch 3/10 - Loss: 0.5049
Epoch 4/10 - Loss: 0.5039
Epoch 5/10 - Loss: 0.5035
Epoch 6/10 - Loss: 0.5034
Epoch 7/10 - Loss: 0.5033
Epoch 8/10 - Loss: 0.5033
Epoch 9/10 - Loss: 0.5033
Epoch 10/10 - Loss: 0.5033


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [7]:
def evaluate(model, dataset, E):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for example in dataset:
            input_vec = average_bekutoru(example["input_ids"], E).float().unsqueeze(0)  # shape: (1, embedding_dim)
            output = model(input_vec)  # shape: (1, 1)
            pred = 1 if output.item() >= 0.5 else 0
            label = int(example["label"].item())

            if pred == label:
                correct += 1
            total += 1

    accuracy = correct / total
    return print(f"Accuracy on dev set: {accuracy * 100:.2f}%")


evaluate(model, dev_dataset, E)

Accuracy on dev set: 70.07%


## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [8]:
def collate(batch):
    # 各入力系列の長さを取得
    lengths = [len(example['input_ids']) for example in batch]

    # 長さの降順にソート
    sorted_batch = sorted(batch, key=lambda x: len(x['input_ids']), reverse=True)

    # 最大長を取得
    max_len = len(sorted_batch[0]['input_ids'])

    # パディングとテンソル化
    padded_input_ids = []
    labels = []

    for example in sorted_batch:
        ids = example['input_ids']
        padded = torch.cat([ids, torch.zeros(max_len - len(ids), dtype=torch.long)])
        padded_input_ids.append(padded)
        labels.append(example['label'])

    # まとめて1つのテンソルに
    input_ids_tensor = torch.stack(padded_input_ids)
    labels_tensor = torch.stack(labels)

    return {
        'input_ids': input_ids_tensor,
        'label': labels_tensor
    }


batch = [train_dataset[i] for i in range(4)]
batch_tensor = collate(batch)
print(batch_tensor)

{'input_ids': tensor([[   13,  6742,    48,  2154,     6, 36257,   646,   872,  3367,    60,
           474,  1747],
        [  949, 14306,  5457,     5,   945,     1,   216,   984,     0,     0,
             0,     0],
        [ 5709,    51, 52777,    26,     1, 13055,  1504,     0,     0,     0,
             0,     0],
        [ 2434,    85, 13026,     2,    92, 26399, 31352,     0,     0,     0,
             0,     0]]), 'label': tensor([[1.],
        [0.],
        [0.],
        [0.]])}


## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [9]:
def batch_average_bekutoru(batch_input_ids, E):
    vectors = []
    for input_ids in batch_input_ids:
        vecs = [torch.tensor(E[i], dtype=torch.float32) for i in input_ids if i != 0]
        if len(vecs) == 0:
            # 万一全て0だった場合（安全対策）
            vecs = [torch.zeros(E.shape[1])]
        avg = torch.mean(torch.stack(vecs), dim=0)
        vectors.append(avg)
    return torch.stack(vectors)

In [16]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

def train_model(model, E, train_dataset, batch_size=16, epochs=10):
    model.train()
    dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)

    loss_fn = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
          optimizer.zero_grad()
          batch_input_ids = batch['input_ids']
          batch_vectors = batch_average_bekutoru(batch_input_ids, E)

          labels = batch['label'].float()  # ラベルを float に
          outputs = model(batch_vectors).float()  # 出力を float に

          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()


        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [14]:
def evaluate(model, dataset, E, batch_size=16):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate)

    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            labels = batch['label']

            features = batch_average_bekutoru(input_ids, E).float()
            outputs = model(features)
            preds = (outputs >= 0.5).float()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")

In [18]:
embedding_dim = E.shape[1]
model = SentimentClassifier(embedding_dim)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# 学習
train_model(model, E, train_dataset, batch_size=16, epochs=10)

# 評価
evaluate(model, dev_dataset, E)

Epoch 1, Loss: 2313.9349
Epoch 2, Loss: 2133.0649
Epoch 3, Loss: 2114.3246
Epoch 4, Loss: 2110.0443
Epoch 5, Loss: 2108.2490
Epoch 6, Loss: 2107.9252
Epoch 7, Loss: 2107.2241
Epoch 8, Loss: 2107.2294
Epoch 9, Loss: 2107.3685
Epoch 10, Loss: 2107.0319
Accuracy: 70.53%


## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。